In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import spectrogram

import tensorflow as tf
import tensorflow.keras.backend as K

from timbral_models import filter_audio_highpass, tf_filter_audio_highpass, timbral_util

%matplotlib inline

In [7]:
from timbral_models import timbral_sharpness, tf_timbral_sharpness

### Loading files

In [14]:
fname = (
    "/home/ubuntu/Documents/code/data/drummer_1_3_sd_001_hits_snare-drum_sticks_x6.wav"
)
fname2 = (
    "/home/ubuntu/Documents/code/data/drummer_3_0_tom_004_hits_low-tom-1_sticks_x5.wav"
)
data_dir = "/home/ubuntu/Documents/code/data/"
audio_samples, fs = timbral_util.file_read(fname, 0, phase_correction=False)
audio_samples2, fs = timbral_util.file_read(fname2, 0, phase_correction=False)
tt = 128 * 128
error = []
grad = True
dev_output = False
audio_samples, fs = timbral_util.file_read(fname, 0, phase_correction=False)[:tt]
audio_samples_t = tf.convert_to_tensor([audio_samples[:tt]], dtype=tf.float32)


N_entire, N_single = timbral_util.specific_loudness(
            audio_samples, Pref=100.0, fs=fs, Mod=0
        )

/home/ubuntu/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/home/ubuntu/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/home/ubuntu/anaconda3/envs/tf_gpu/lib/python3.8/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


In [37]:
def sharpness_Fastl(loudspec):
    """
      Calculates the sharpness based on FASTL (1991)
      Expression for weighting function obtained by fitting an
      equation to data given in 'Psychoacoustics: Facts and Models'
      using MATLAB basic fitting function
      Original Matlab code by Claire Churchill Sep 2004
      Transcoded by Andy Pearce 2018
    """
    n = len(loudspec)
    gz = np.ones(140)
    z = np.arange(141, n + 1)
    gzz = (
        0.00012 * (z / 10.0) ** 4
        - 0.0056 * (z / 10.0) ** 3
        + 0.1 * (z / 10.0) ** 2
        - 0.81 * (z / 10.0)
        + 3.5
    )
    gz = np.concatenate((gz, gzz))
    z = np.arange(0.1, n / 10.0 + 0.1, 0.1)

    sharp = 0.11 * np.sum(loudspec * gz * z * 0.1) / np.sum(loudspec * 0.1)
    return sharp


def tf_sharpness_Fastl(loudspec):
    """
      Calculates the sharpness based on FASTL (1991)
      Expression for weighting function obtained by fitting an
      equation to data given in 'Psychoacoustics: Facts and Models'
      using MATLAB basic fitting function
      Original Matlab code by Claire Churchill Sep 2004
      Transcoded by Andy Pearce 2018
    """
    n = loudspec.shape[-1]
    gz = tf.ones(140, dtype = loudspec.dtype)
    z = K.arange(141, n + 1, dtype = gz.dtype)
    gzz = (
        0.00012 * (z / 10.0) ** 4
        - 0.0056 * (z / 10.0) ** 3
        + 0.1 * (z / 10.0) ** 2
        - 0.81 * (z / 10.0)
        + 3.5
    )
    gz = tf.concat((gz, gzz), axis = -1)
    z = np.arange(0.1, n / 10.0 + 0.1, 0.1)

    sharp = 0.11 * K.sum(loudspec * gz * z * 0.1, axis = -1) / K.sum(loudspec * 0.1, axis = -1)
    return sharp
print(sharpness_Fastl(N_single), tf_sharpness_Fastl(N_single))
print("erreur ::", abs(sharpness_Fastl(N_single)- tf_sharpness_Fastl(N_single).numpy()) / sharpness_Fastl(N_single) * 100, "%")

1.2782242790211005 tf.Tensor(1.2782242790211007, shape=(), dtype=float64)
erreur :: 1.73713336985806e-14 %


### Loudness

In [38]:
def specific_loudness(x, Pref, fs, Mod):
    """
      Calculates loudness in 3rd octave bands
        based on ISO 532 B / DIN 45631
        Source: BASIC code in J Acoust Soc Jpn(E) 12, 1(1991)
        x = signal
        Pref = refernce value[dB]
        fs = sampling frequency[Hz]
        Mod = 0 for free field
        Mod = 1 for diffuse field

        Returns
        N_entire = entire loudness[sone]
        N_single = partial loudness[sone / Bark]

        Original Matlab code by Claire Churchill Jun. 2004
        Transcoded by Andy Pearce 2018
    """

    # 'Generally used third-octave band filters show a leakage towards neighbouring filters of about -20 dB. This
    # means that a 70dB, 1 - kHz tone produces the following levels at different centre
    # frequencies: 10dB at 500Hz, 30dB at 630Hz, 50dB at 800Hz and 70dB at 1kHz.
    # P211 Psychoacoustics: Facts and Models, E.Zwicker and H.Fastl
    # (A filter order of 4 gives approx this result)

    # set default
    Fmin = 25
    Fmax = 12500
    order = 4
    # filter the audio
    Ptotal, P, F = filter_third_octaves_downsample(x, Pref, fs, Fmin, Fmax, order)

    # set more defaults for perceptual filters

    # Centre frequencies of 1 / 3 Oct bands(FR)
    FR = np.array(
        [
            25,
            31.5,
            40,
            50,
            63,
            80,
            100,
            125,
            160,
            200,
            250,
            315,
            400,
            500,
            630,
            800,
            1000,
            1250,
            1600,
            2000,
            2500,
            3150,
            4000,
            5000,
            6300,
            8000,
            10000,
            12500,
        ]
    )

    # Ranges of 1 / 3 Oct bands for correction at low frequencies according to equal loudness contours
    RAP = np.array([45, 55, 65, 71, 80, 90, 100, 120])

    # Reduction of 1/3 Oct Band levels at low frequencies according to equal loudness contours
    # within the eight ranges defined by RAP(DLL)
    DLL = np.array(
        [
            [-32, -24, -16, -10, -5, 0, -7, -3, 0, -2, 0],
            [-29, -22, -15, -10, -4, 0, -7, -2, 0, -2, 0],
            [-27, -19, -14, -9, -4, 0, -6, -2, 0, -2, 0],
            [-25, -17, -12, -9, -3, 0, -5, -2, 0, -2, 0],
            [-23, -16, -11, -7, -3, 0, -4, -1, 0, -1, 0],
            [-20, -14, -10, -6, -3, 0, -4, -1, 0, -1, 0],
            [-18, -12, -9, -6, -2, 0, -3, -1, 0, -1, 0],
            [-15, -10, -8, -4, -2, 0, -3, -1, 0, -1, 0],
        ]
    )

    # Critical band level at absolute threshold without taking into account the
    # transmission characteristics of the ear
    # Threshold due to internal noise
    LTQ = np.array([30, 18, 12, 8, 7, 6, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
    # Hearing thresholds for the excitation levels (each number corresponds to a critical band 12.5kHz is not included)

    # Attenuation representing transmission between freefield and our hearing system
    A0 = np.array(
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.5, -1.6, -3.2, -5.4, -5.6, -4, -1.5, 2, 5, 12]
    )
    # Attenuation due to transmission in the middle ear
    # Moore et al disagrees with this being flat for low frequencies

    # Level correction to convert from a free field to a diffuse field(last critical band 12.5 kHz is not included)
    DDF = np.array(
        [
            0,
            0,
            0.5,
            0.9,
            1.2,
            1.6,
            2.3,
            2.8,
            3,
            2,
            0,
            -1.4,
            -2,
            -1.9,
            -1,
            0.5,
            3,
            4,
            4.3,
            4,
        ]
    )

    # Correction factor because using third octave band levels(rather than critical bands)
    DCB = np.array(
        [
            -0.25,
            -0.6,
            -0.8,
            -0.8,
            -0.5,
            0,
            0.5,
            1.1,
            1.5,
            1.7,
            1.8,
            1.8,
            1.7,
            1.6,
            1.4,
            1.2,
            0.8,
            0.5,
            0,
            -0.5,
        ]
    )

    # Upper limits of the approximated critical bands
    ZUP = np.array(
        [
            0.9,
            1.8,
            2.8,
            3.5,
            4.4,
            5.4,
            6.6,
            7.9,
            9.2,
            10.6,
            12.3,
            13.8,
            15.2,
            16.7,
            18.1,
            19.3,
            20.6,
            21.8,
            22.7,
            23.6,
            24,
        ]
    )

    # Range of specific loudness for the determination of the steepness of the upper slopes in the specific loudness
    # - critical band rate pattern(used to plot the correct USL curve)
    RNS = np.array(
        [
            21.5,
            18,
            15.1,
            11.5,
            9,
            6.1,
            4.4,
            3.1,
            2.13,
            1.36,
            0.82,
            0.42,
            0.30,
            0.22,
            0.15,
            0.10,
            0.035,
            0,
        ]
    )

    # This is used to design the right hand slope of the loudness
    USL = np.array(
        [
            [13.0, 8.2, 6.3, 5.5, 5.5, 5.5, 5.5, 5.5],
            [9.0, 7.5, 6.0, 5.1, 4.5, 4.5, 4.5, 4.5],
            [7.8, 6.7, 5.6, 4.9, 4.4, 3.9, 3.9, 3.9],
            [6.2, 5.4, 4.6, 4.0, 3.5, 3.2, 3.2, 3.2],
            [4.5, 3.8, 3.6, 3.2, 2.9, 2.7, 2.7, 2.7],
            [3.7, 3.0, 2.8, 2.35, 2.2, 2.2, 2.2, 2.2],
            [2.9, 2.3, 2.1, 1.9, 1.8, 1.7, 1.7, 1.7],
            [2.4, 1.7, 1.5, 1.35, 1.3, 1.3, 1.3, 1.3],
            [1.95, 1.45, 1.3, 1.15, 1.1, 1.1, 1.1, 1.1],
            [1.5, 1.2, 0.94, 0.86, 0.82, 0.82, 0.82, 0.82],
            [0.72, 0.67, 0.64, 0.63, 0.62, 0.62, 0.62, 0.62],
            [0.59, 0.53, 0.51, 0.50, 0.42, 0.42, 0.42, 0.42],
            [0.40, 0.33, 0.26, 0.24, 0.24, 0.22, 0.22, 0.22],
            [0.27, 0.21, 0.20, 0.18, 0.17, 0.17, 0.17, 0.17],
            [0.16, 0.15, 0.14, 0.12, 0.11, 0.11, 0.11, 0.11],
            [0.12, 0.11, 0.10, 0.08, 0.08, 0.08, 0.08, 0.08],
            [0.09, 0.08, 0.07, 0.06, 0.06, 0.06, 0.06, 0.05],
            [0.06, 0.05, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02],
        ]
    )

    # apply weighting factors
    Xp = np.zeros(11)
    Ti = np.zeros(11)
    for i in range(11):
        j = 0
        while (P[i] > (RAP[j] - DLL[j, i])) & (j < 7):
            j += 1
        Xp[i] = P[i] + DLL[j, i]
        Ti[i] = 10.0 ** (Xp[i] / 10.0)

    # Intensity values in first three critical bands calculated
    Gi = np.zeros(3)
    # Gi(1) is the first critical band (sum of two octaves(25Hz to 80Hz))
    Gi[0] = np.sum(Ti[0:6])
    # Gi(2) is the second critical band (sum of octave(100Hz to 160Hz))
    Gi[1] = np.sum(Ti[6:9])
    # Gi(3) is the third critical band (sum of two third octave bands(200Hz to 250Hz))
    Gi[2] = np.sum(Ti[9:11])

    if np.max(Gi) > 0.0:
        FNGi = 10 * np.log10(Gi)
    else:
        FNGi = -1.0 * np.inf
    LCB = np.zeros_like(Gi)
    for i in range(3):
        if Gi[i] > 0:
            LCB[i] = FNGi[i]
        else:
            LCB[i] = 0

    # Calculate the main loudness in each critical band
    Le = np.ones(20)
    Lk = np.ones_like(Le)
    Nm = np.ones(21)
    # print("timbarl_loudness P::", P)

    for i in range(20):
        Le[i] = P[i + 8]
        if i <= 2:
            Le[i] = LCB[i]
        Lk[i] = Le[i] - A0[i]
        Nm[i] = 0
        if Mod == 1:
            Le[i] = Le[i] + DDF[i]
        if Le[i] > LTQ[i]:
            Le[i] = Lk[i] - DCB[i]
            S = 0.25
            MP1 = 0.0635 * 10.0 ** (0.025 * LTQ[i])
            MP2 = (1 - S + S * 10 ** (0.1 * (Le[i] - LTQ[i]))) ** 0.25 - 1
            Nm[i] = MP1 * MP2
            if Nm[i] <= 0:
                Nm[i] = 0
    Nm[20] = 0

    KORRY = 0.4 + 0.32 * Nm[0] ** 0.2
    if KORRY > 1:
        KORRY = 1

    Nm[0] = Nm[0] * KORRY

    # Add masking curves to the main loudness in each third octave band
    N = 0
    z1 = 0  # critical band rate starts at 0
    n1 = 0  # loudness level starts at 0
    j = 17
    iz = 0
    z = 0.1
    ns = []

    for i in range(21):
        # Determines where to start on the slope
        ig = i - 1
        if ig > 7:
            ig = 7
        control = 1
        # ZUP is the upper limit of the approximated critical band
        while (z1 < ZUP[i]) | (control == 1):
            # Determines which of the slopes to use
            if n1 < Nm[i]:  # Nm is the main loudness level
                j = 0
                while RNS[j] > Nm[i]:  # the value of j is used below to build a slope
                    # j becomes the index at which Nm(i) is first greater than RNS
                    j += 1

            # The flat portions of the loudness graph
            if n1 <= Nm[i]:
                z2 = ZUP[i]  # z2 becomes the upper limit of the critical band
                n2 = Nm[i]
                N = N + n2 * (z2 - z1)  # Sums the output(N_entire)
                for k in np.arange(z, z2 + 0.01, 0.1):
                    if not ns:
                        ns.append(n2)
                    else:
                        if iz == len(ns):
                            ns.append(n2)
                        elif iz < len(ns):
                            ns[iz] = n2

                    if k < (z2 - 0.05):
                        iz += 1
                z = k  # z becomes the last value of k
                z = round(z * 10) * 0.1

            # The sloped portions of the loudness graph
            if n1 > Nm[i]:
                n2 = RNS[j]
                if n2 < Nm[i]:
                    n2 = Nm[i]
                dz = (n1 - n2) / USL[j, ig]  # USL = slopes
                dz = round(dz * 10) * 0.1
                if dz == 0:
                    dz = 0.1
                z2 = z1 + dz
                if z2 > ZUP[i]:
                    z2 = ZUP[i]
                    dz = z2 - z1
                    n2 = n1 - dz * USL[j, ig]  # USL = slopes
                N = N + dz * (n1 + n2) / 2.0  # Sums the output(N_entire)
                for k in np.arange(z, z2 + 0.01, 0.1):
                    if not ns:
                        ns.append(n1 - (k - z1) * USL[j, ig])
                    else:
                        if iz == len(ns):
                            ns.append(n1 - (k - z1) * USL[j, ig])
                        elif iz < len(ns):
                            ns[iz] = n1 - (k - z1) * USL[j, ig]
                    if k < (z2 - 0.05):
                        iz += 1
                z = k
                z = round(z * 10) * 0.1
            if n2 == RNS[j]:
                j += 1
            if j > 17:
                j = 17
            n1 = n2
            z1 = z2
            z1 = round(z1 * 10) * 0.1
            control += 1

    if N < 0:
        N = 0

    if N <= 16:
        N = np.floor(N * 1000 + 0.5) / 1000.0
    else:
        N = np.floor(N * 100 + 0.05) / 100.0

    LN = 40.0 * (N + 0.0005) ** 0.35

    if LN < 3:
        LN = 3

    if N >= 1:
        LN = 10 * np.log10(N) / np.log10(2) + 40

    N_single = np.zeros(240)
    for i in range(240):
        N_single[i] = ns[i]

    N_entire = N
    return N_entire, N_single

In [39]:
def tf_specific_loudness(x, Pref, fs, Mod):
    """
      Calculates loudness in 3rd octave bands
        based on ISO 532 B / DIN 45631
        Source: BASIC code in J Acoust Soc Jpn(E) 12, 1(1991)
        x = signal
        Pref = refernce value[dB]
        fs = sampling frequency[Hz]
        Mod = 0 for free field
        Mod = 1 for diffuse field

        Returns
        N_entire = entire loudness[sone]
        N_single = partial loudness[sone / Bark]

        Original Matlab code by Claire Churchill Jun. 2004
        Transcoded by Andy Pearce 2018
    """

    # 'Generally used third-octave band filters show a leakage towards neighbouring filters of about -20 dB. This
    # means that a 70dB, 1 - kHz tone produces the following levels at different centre
    # frequencies: 10dB at 500Hz, 30dB at 630Hz, 50dB at 800Hz and 70dB at 1kHz.
    # P211 Psychoacoustics: Facts and Models, E.Zwicker and H.Fastl
    # (A filter order of 4 gives approx this result)

    # set default
    Fmin = 25
    Fmax = 12500
    order = 4
    # filter the audio
    Ptotal, P, F = tf_filter_third_octaves_downsample(x, Pref, fs, Fmin, Fmax, order)

    # set more defaults for perceptual filters

    # Centre frequencies of 1 / 3 Oct bands(FR)
    FR = np.array(
        [
            25,
            31.5,
            40,
            50,
            63,
            80,
            100,
            125,
            160,
            200,
            250,
            315,
            400,
            500,
            630,
            800,
            1000,
            1250,
            1600,
            2000,
            2500,
            3150,
            4000,
            5000,
            6300,
            8000,
            10000,
            12500,
        ]
    )

    # Ranges of 1 / 3 Oct bands for correction at low frequencies according to equal loudness contours
    RAP = np.array([45, 55, 65, 71, 80, 90, 100, 120])

    # Reduction of 1/3 Oct Band levels at low frequencies according to equal loudness contours
    # within the eight ranges defined by RAP(DLL)
    DLL = np.array(
        [
            [-32, -24, -16, -10, -5, 0, -7, -3, 0, -2, 0],
            [-29, -22, -15, -10, -4, 0, -7, -2, 0, -2, 0],
            [-27, -19, -14, -9, -4, 0, -6, -2, 0, -2, 0],
            [-25, -17, -12, -9, -3, 0, -5, -2, 0, -2, 0],
            [-23, -16, -11, -7, -3, 0, -4, -1, 0, -1, 0],
            [-20, -14, -10, -6, -3, 0, -4, -1, 0, -1, 0],
            [-18, -12, -9, -6, -2, 0, -3, -1, 0, -1, 0],
            [-15, -10, -8, -4, -2, 0, -3, -1, 0, -1, 0],
        ]
    )

    # Critical band level at absolute threshold without taking into account the
    # transmission characteristics of the ear
    # Threshold due to internal noise
    LTQ = np.array([30, 18, 12, 8, 7, 6, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
    # Hearing thresholds for the excitation levels (each number corresponds to a critical band 12.5kHz is not included)

    # Attenuation representing transmission between freefield and our hearing system
    A0 = np.array(
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.5, -1.6, -3.2, -5.4, -5.6, -4, -1.5, 2, 5, 12]
    )
    # Attenuation due to transmission in the middle ear
    # Moore et al disagrees with this being flat for low frequencies

    # Level correction to convert from a free field to a diffuse field(last critical band 12.5 kHz is not included)
    DDF = np.array(
        [
            0,
            0,
            0.5,
            0.9,
            1.2,
            1.6,
            2.3,
            2.8,
            3,
            2,
            0,
            -1.4,
            -2,
            -1.9,
            -1,
            0.5,
            3,
            4,
            4.3,
            4,
        ]
    )

    # Correction factor because using third octave band levels(rather than critical bands)
    DCB = np.array(
        [
            -0.25,
            -0.6,
            -0.8,
            -0.8,
            -0.5,
            0,
            0.5,
            1.1,
            1.5,
            1.7,
            1.8,
            1.8,
            1.7,
            1.6,
            1.4,
            1.2,
            0.8,
            0.5,
            0,
            -0.5,
        ]
    )

    # Upper limits of the approximated critical bands
    ZUP = np.array(
        [
            0.9,
            1.8,
            2.8,
            3.5,
            4.4,
            5.4,
            6.6,
            7.9,
            9.2,
            10.6,
            12.3,
            13.8,
            15.2,
            16.7,
            18.1,
            19.3,
            20.6,
            21.8,
            22.7,
            23.6,
            24,
        ]
    )

    # Range of specific loudness for the determination of the steepness of the upper slopes in the specific loudness
    # - critical band rate pattern(used to plot the correct USL curve)
    RNS = np.array(
        [
            21.5,
            18,
            15.1,
            11.5,
            9,
            6.1,
            4.4,
            3.1,
            2.13,
            1.36,
            0.82,
            0.42,
            0.30,
            0.22,
            0.15,
            0.10,
            0.035,
            0,
        ]
    )

    # This is used to design the right hand slope of the loudness
    USL = np.array(
        [
            [13.0, 8.2, 6.3, 5.5, 5.5, 5.5, 5.5, 5.5],
            [9.0, 7.5, 6.0, 5.1, 4.5, 4.5, 4.5, 4.5],
            [7.8, 6.7, 5.6, 4.9, 4.4, 3.9, 3.9, 3.9],
            [6.2, 5.4, 4.6, 4.0, 3.5, 3.2, 3.2, 3.2],
            [4.5, 3.8, 3.6, 3.2, 2.9, 2.7, 2.7, 2.7],
            [3.7, 3.0, 2.8, 2.35, 2.2, 2.2, 2.2, 2.2],
            [2.9, 2.3, 2.1, 1.9, 1.8, 1.7, 1.7, 1.7],
            [2.4, 1.7, 1.5, 1.35, 1.3, 1.3, 1.3, 1.3],
            [1.95, 1.45, 1.3, 1.15, 1.1, 1.1, 1.1, 1.1],
            [1.5, 1.2, 0.94, 0.86, 0.82, 0.82, 0.82, 0.82],
            [0.72, 0.67, 0.64, 0.63, 0.62, 0.62, 0.62, 0.62],
            [0.59, 0.53, 0.51, 0.50, 0.42, 0.42, 0.42, 0.42],
            [0.40, 0.33, 0.26, 0.24, 0.24, 0.22, 0.22, 0.22],
            [0.27, 0.21, 0.20, 0.18, 0.17, 0.17, 0.17, 0.17],
            [0.16, 0.15, 0.14, 0.12, 0.11, 0.11, 0.11, 0.11],
            [0.12, 0.11, 0.10, 0.08, 0.08, 0.08, 0.08, 0.08],
            [0.09, 0.08, 0.07, 0.06, 0.06, 0.06, 0.06, 0.05],
            [0.06, 0.05, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02],
        ]
    )

    # apply weighting factors
    Xp = np.zeros(11)
    Ti = np.zeros(11)
    for i in range(11):
        j = 0
        while (P[i] > (RAP[j] - DLL[j, i])) & (j < 7):
            j += 1
        Xp[i] = P[i] + DLL[j, i]
        Ti[i] = 10.0 ** (Xp[i] / 10.0)

    # Intensity values in first three critical bands calculated
    Gi = np.zeros(3)
    # Gi(1) is the first critical band (sum of two octaves(25Hz to 80Hz))
    Gi[0] = np.sum(Ti[0:6])
    # Gi(2) is the second critical band (sum of octave(100Hz to 160Hz))
    Gi[1] = np.sum(Ti[6:9])
    # Gi(3) is the third critical band (sum of two third octave bands(200Hz to 250Hz))
    Gi[2] = np.sum(Ti[9:11])

    if np.max(Gi) > 0.0:
        FNGi = 10 * np.log10(Gi)
    else:
        FNGi = -1.0 * np.inf
    LCB = np.zeros_like(Gi)
    for i in range(3):
        if Gi[i] > 0:
            LCB[i] = FNGi[i]
        else:
            LCB[i] = 0

    # Calculate the main loudness in each critical band
    Le = np.ones(20)
    Lk = np.ones_like(Le)
    Nm = np.ones(21)
    for i in range(20):
        Le[i] = P[i + 8]
        if i <= 2:
            Le[i] = LCB[i]
        Lk[i] = Le[i] - A0[i]
        Nm[i] = 0
        if Mod == 1:
            Le[i] = Le[i] + DDF[i]
        if Le[i] > LTQ[i]:
            Le[i] = Lk[i] - DCB[i]
            S = 0.25
            MP1 = 0.0635 * 10.0 ** (0.025 * LTQ[i])
            MP2 = (1 - S + S * 10 ** (0.1 * (Le[i] - LTQ[i]))) ** 0.25 - 1
            Nm[i] = MP1 * MP2
            if Nm[i] <= 0:
                Nm[i] = 0
    Nm[20] = 0

    KORRY = 0.4 + 0.32 * Nm[0] ** 0.2
    if KORRY > 1:
        KORRY = 1

    Nm[0] = Nm[0] * KORRY

    # Add masking curves to the main loudness in each third octave band
    N = 0
    z1 = 0  # critical band rate starts at 0
    n1 = 0  # loudness level starts at 0
    j = 17
    iz = 0
    z = 0.1
    ns = []

    for i in range(21):
        # Determines where to start on the slope
        ig = i - 1
        if ig > 7:
            ig = 7
        control = 1
        # ZUP is the upper limit of the approximated critical band
        while (z1 < ZUP[i]) | (control == 1):
            # Determines which of the slopes to use
            if n1 < Nm[i]:  # Nm is the main loudness level
                j = 0
                while RNS[j] > Nm[i]:  # the value of j is used below to build a slope
                    # j becomes the index at which Nm(i) is first greater than RNS
                    j += 1

            # The flat portions of the loudness graph
            if n1 <= Nm[i]:
                z2 = ZUP[i]  # z2 becomes the upper limit of the critical band
                n2 = Nm[i]
                N = N + n2 * (z2 - z1)  # Sums the output(N_entire)
                for k in np.arange(z, z2 + 0.01, 0.1):
                    if not ns:
                        ns.append(n2)
                    else:
                        if iz == len(ns):
                            ns.append(n2)
                        elif iz < len(ns):
                            ns[iz] = n2

                    if k < (z2 - 0.05):
                        iz += 1
                z = k  # z becomes the last value of k
                z = round(z * 10) * 0.1

            # The sloped portions of the loudness graph
            if n1 > Nm[i]:
                n2 = RNS[j]
                if n2 < Nm[i]:
                    n2 = Nm[i]
                dz = (n1 - n2) / USL[j, ig]  # USL = slopes
                dz = round(dz * 10) * 0.1
                if dz == 0:
                    dz = 0.1
                z2 = z1 + dz
                if z2 > ZUP[i]:
                    z2 = ZUP[i]
                    dz = z2 - z1
                    n2 = n1 - dz * USL[j, ig]  # USL = slopes
                N = N + dz * (n1 + n2) / 2.0  # Sums the output(N_entire)
                for k in np.arange(z, z2 + 0.01, 0.1):
                    if not ns:
                        ns.append(n1 - (k - z1) * USL[j, ig])
                    else:
                        if iz == len(ns):
                            ns.append(n1 - (k - z1) * USL[j, ig])
                        elif iz < len(ns):
                            ns[iz] = n1 - (k - z1) * USL[j, ig]
                    if k < (z2 - 0.05):
                        iz += 1
                z = k
                z = round(z * 10) * 0.1
            if n2 == RNS[j]:
                j += 1
            if j > 17:
                j = 17
            n1 = n2
            z1 = z2
            z1 = round(z1 * 10) * 0.1
            control += 1

    if N < 0:
        N = 0

    if N <= 16:
        N = np.floor(N * 1000 + 0.5) / 1000.0
    else:
        N = np.floor(N * 100 + 0.05) / 100.0

    LN = 40.0 * (N + 0.0005) ** 0.35

    if LN < 3:
        LN = 3

    if N >= 1:
        LN = 10 * np.log10(N) / np.log10(2) + 40

    N_single = np.zeros(240)
    for i in range(240):
        N_single[i] = ns[i]

    N_entire = N
    return N_entire, N_single



### RMS on windowed

In [43]:
windowed_audio = timbral_util.tf_window_audio(audio_samples_t, window_length=4096)
# windowed_audio = tf.signal.frame(audio_samples, 4096, 4096, pad_end=True)

windowed_sharpness = []
windowed_rms = []
print(windowed_audio.shape)
for i in range(windowed_audio.shape[0]):
    samples = windowed_audio[i, :]

    # calculate the rms and append to list
    windowed_rms.append(K.sqrt(K.mean(samples * samples)))

(1, 4, 4096)


In [44]:
windowed_audio

<tf.Tensor: shape=(1, 4, 4096), dtype=float32, numpy=
array([[[ 7.6417542e-05,  1.5283508e-04,  7.6417542e-05, ...,
          5.3874371e-03, -5.7313161e-04, -8.7115997e-03],
        [-1.9104386e-02, -2.4568241e-02, -2.1740792e-02, ...,
          3.7291761e-02,  3.6374751e-02,  3.5037443e-02],
        [ 3.3547301e-02,  3.1636864e-02,  2.9650008e-02, ...,
          2.7128228e-03,  2.2161088e-03,  1.6429772e-03],
        [ 8.0238422e-04,  0.0000000e+00, -6.4954913e-04, ...,
          5.4638544e-03,  5.1199757e-03,  4.6232613e-03]]], dtype=float32)>

In [48]:
K.sqrt(K.mean(windowed_audio * windowed_audio, axis = -1))

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.19183034, 0.02910141, 0.01238957, 0.00605544]], dtype=float32)>

<tf.Tensor: shape=(4, 4096), dtype=float32, numpy=
array([[ 7.6417542e-05,  1.5283508e-04,  7.6417542e-05, ...,
         5.3874371e-03, -5.7313161e-04, -8.7115997e-03],
       [-1.9104386e-02, -2.4568241e-02, -2.1740792e-02, ...,
         3.7291761e-02,  3.6374751e-02,  3.5037443e-02],
       [ 3.3547301e-02,  3.1636864e-02,  2.9650008e-02, ...,
         2.7128228e-03,  2.2161088e-03,  1.6429772e-03],
       [ 8.0238422e-04,  0.0000000e+00, -6.4954913e-04, ...,
         5.4638544e-03,  5.1199757e-03,  4.6232613e-03]], dtype=float32)>